In [1]:
import pandas
import datetime as dt

In [2]:
df = pandas.read_parquet("clean_retail.parquet")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397925 entries, 0 to 397924
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      397925 non-null  object        
 1   StockCode    397925 non-null  object        
 2   Description  397925 non-null  object        
 3   Quantity     397925 non-null  int64         
 4   InvoiceDate  397925 non-null  datetime64[ns]
 5   Price        397925 non-null  float64       
 6   Customer ID  397925 non-null  float64       
 7   Country      397925 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [4]:
df.head(3)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In [5]:
df['Total'] = df['Quantity'] * df['Price']

In [6]:
snapshot_date = df["InvoiceDate"].max() + dt.timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [7]:
customer = df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (snapshot_date - max(x)).days,
    "Invoice": "nunique",
    "Total": "sum",
    "Price": "mean",
    "StockCode": "nunique",
    "Quantity": "mean"
}).rename(columns={
    "InvoiceDate": "Recency",
    "Invoice": "Frequency",
    "Total": "Monetary",
    "StockCode": "UniqueItems",
    "Price": "AvgUnitPrice",
    "Quantity": "AvgQuantPerOrder"
})

In [8]:
customer.head(3)

,Recency,Frequency,Monetary,AvgUnitPrice,UniqueItems,AvgQuantPerOrder
Customer ID,,,,,,
12346.0,326,1,77183.60,1.040000,1,74215.000000
12347.0,2,7,4310.00,2.644011,103,13.505495
12348.0,75,4,1797.24,5.764839,22,75.516129


In [9]:
customer.shape

(4339, 6)

In [10]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4339 entries, 12346.0 to 18287.0
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Recency           4339 non-null   int64  
 1   Frequency         4339 non-null   int64  
 2   Monetary          4339 non-null   float64
 3   AvgUnitPrice      4339 non-null   float64
 4   UniqueItems       4339 non-null   int64  
 5   AvgQuantPerOrder  4339 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 237.3 KB


In [11]:
customer.to_csv("customer_features.csv")